In [1]:
import ibm_boto3
from ibm_botocore.client import Config

In [2]:
#If you are using HMAC authentication
#aws_access_key_id = 'keyid' #Platform API Key: https://console.bluemix.net/iam/#/apikeys
#aws_secret_access_key = 'secretkey'
func_id_api_key = 'funcid'
auth_endpoint = 'https://iam.bluemix.net/oidc/token' # You don't have to change this
# Only service_endpoint identifies the COS service, service instance_id is OPTIONAL.
service_instance_id = 'instanceid'
service_endpoint = 'https://s3.us-south.objectstorage.softlayer.net'

In [3]:
cos_resource = ibm_boto3.resource('s3',
                      #If you are using HMAC authentication            
                      #aws_access_key_id=aws_access_key_id, 
                      #aws_secret_access_key=aws_secret_access_key,
                      ibm_api_key_id=func_id_api_key, #api key of the owner
                      ibm_service_instance_id=service_instance_id,#"crn:v1:bluemix:public:cloud-object-storage:global:a/bbec8f6fff5d44598261ffb8abdc1889:758e3fa9-fa50-4b8a-95e8-31b05d4891b7::", #resources instance id, OPTIONAL
                      ibm_auth_endpoint=auth_endpoint,
                      config=Config(signature_version='oauth'),
                      endpoint_url=service_endpoint)

In [4]:
def build_path(l):
    # Given ['a', 'b', 'c']
    # Output ['a', 'a/b', 'a/b/c']
    k = []
    for i in l:
        if len(k) == 0:
            k.append(i)
        else:
            t = k[-1] + '/' + i
            k.append(t)
    return k

In [5]:
build_path(['a', 'b', 'c'])

['a', 'a/b', 'a/b/c']

In [6]:
to_be_deleted = []
for o in cos_resource.Bucket('lotame-landing-zone').objects.all():
    if o.key.endswith('_SUCCESS'):
        l = o.key.split('/')
        to_be_deleted += build_path(l)

In [7]:
to_be_deleted

['dmp-ref-dmp-aud',
 'dmp-ref-dmp-aud/_SUCCESS',
 'dmp_upid_dsp_xref',
 'dmp_upid_dsp_xref/current',
 'dmp_upid_dsp_xref/current/_SUCCESS',
 'tmp',
 'tmp/dmp_ref_dmp_aud',
 'tmp/dmp_ref_dmp_aud/_SUCCESS']

In [8]:
for o in cos_resource.Bucket('lotame-landing-zone').objects.all():
    if o.key in to_be_deleted:
        print(o)

s3.ObjectSummary(bucket_name='lotame-landing-zone', key='dmp-ref-dmp-aud')
s3.ObjectSummary(bucket_name='lotame-landing-zone', key='dmp-ref-dmp-aud/_SUCCESS')
s3.ObjectSummary(bucket_name='lotame-landing-zone', key='dmp_upid_dsp_xref/current')
s3.ObjectSummary(bucket_name='lotame-landing-zone', key='dmp_upid_dsp_xref/current/_SUCCESS')
s3.ObjectSummary(bucket_name='lotame-landing-zone', key='tmp/dmp_ref_dmp_aud')
s3.ObjectSummary(bucket_name='lotame-landing-zone', key='tmp/dmp_ref_dmp_aud/_SUCCESS')


In [9]:
# Uncomment following lines when you are ready to delete
# for o in cos_resource.Bucket('lotame-landing-zone').objects.all():
#     if o.key in to_be_deleted:
#         o.delete()